In [ ]:
import librosa
import librosa.display
import IPython
import os
import numpy as np
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**LOADING THE DATA**

The training set consists of 6 classes and 20 samples from each of the two validation classes. The validation set consists of these two classes except the 20 samples 

In [ ]:
file="/content/drive/MyDrive/urbansound8k/cnn_train"
files_in_directory = os.listdir(file) #EDIT THIS FOR ANOTHER DIRECTORY
examples = [file for file in files_in_directory]

print(examples)
XXtrain=[]
YYtrain=[]
from matplotlib import pyplot as plt
for e in range(len(examples)-1):
  path=file+"/"+"class_"+str(e)
  f=os.listdir(path)
  # print(path)
  print(e)
  for foo in f:
    # print(foo)
    jk=np.array(Image.open(path+"/"+foo))
    XXtrain.append(jk)
    YYtrain.append(e)
lxtrain=np.array(XXtrain)
lytrain=np.array(YYtrain)

In [ ]:
! rm -r /content/drive/MyDrive/urbansound8k/cnn_eval/.ipynb_checkpoints

In [ ]:
np.save('/content/drive/MyDrive/urbansound8k/cnn_train/lxtrain', lxtrain )
np.save('/content/drive/MyDrive/urbansound8k/cnn_train/lytrain', lytrain )

In [ ]:
! rm -r /content/drive/MyDrive/urbansound8k/cnn_train/class_0/.ipynb_checkpoints

In [ ]:
file="/content/drive/MyDrive/urbansound8k/cnn_eval"
files_in_directory = os.listdir(file)#EDIT THIS FOR ANOTHER DIRECTORY
examples = [file for file in files_in_directory]
XXtest=[]
YYtest=[]
print((examples))
from matplotlib import pyplot as plt
for e in range(len(examples)):
  path=file+"/"+"class_"+str(e+6)
  f=os.listdir(path)
  for foo in f:
    jk=np.array(Image.open(path+"/"+foo))
    XXtest.append(jk)
    YYtest.append(e+6)
lxtest=np.array(XXtest)
lytest=np.array(YYtest)

In [ ]:
lytest.shape

(1264,)

In [ ]:
np.save('/content/drive/MyDrive/urbansound8k/cnn_eval/lxtest', lxtest )

In [ ]:
np.save('/content/drive/MyDrive/urbansound8k/cnn_eval/lytest', lytest )

**DEFINING THE MODEL**

In [ ]:
model = Sequential()
model.add(layers.Conv2D(32, 7, strides =(4,4), padding="same", input_shape=(120, 128, 4)))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((4,4)))
model.add(layers.Conv2D(128, 4, padding="same"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))



**TRAINING AND EVALUATION**

The evaluation criteria used for the model is the best accuracy obtained on the validation set


In [ ]:
### WE USE EVALUATION CRITERIA AS THE BEST ACCURACY OBTAINED ON VALIDATION SET
adam = Adam(learning_rate=0.0003)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', weighted_metrics=['acc'])

In [ ]:
es = EarlyStopping(monitor='val_loss', verbose=10, patience=50)
mc = ModelCheckpoint('/content/drive/MyDrive/urbansound8k/cnn_6.h5', monitor='val_loss', save_best_only=True, verbose=1)

In [ ]:
model_history = model.fit(lxtrain, lytrain, validation_data=(lxtest, lytest), epochs=30, callbacks=[es, mc], workers=32, max_queue_size=20)

In [ ]:
ypred = np.argmax(model.predict(lxtest), axis=-1)

In [ ]:
df = pd.read_csv("eval.csv")
df["ypred"] = ypred
df.to_csv("eval.csv", index=False)

In [ ]:
model.summary()